In [2]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix, f1_score

# Set the path to your dataset
train_dir = '/data/dataset'
test_dir = '/data/dataset'

# Set the image size and batch size
img_size = (299, 299)
batch_size = 32

# Create a data generator for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Add a new classification layer
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)

predictions = tf.keras.layers.Dense(6, activation='softmax')(x)

# Combine the base model and classification layer
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Save the model
model.save('/code/model/ResNet50.h5')
labels = ['Foi_Thong', 'Hang_Kra_Rog_Phu_Phan_ST1', 'Hang_Suea_Sakonnakhon_TT1', 'Kroeng_Krawia', 'Tanao_Si_Kan_Khaw_WA1', 'Tanao_Si_Kan_Sang_RD1']

# Train the model and record validation loss and accuracy after each epoch
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

print("\n")

# Generate predictions for the test set
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Calculate the confusion matrix and print it
confusion_matrix = confusion_matrix(test_generator.classes, y_pred)
print("Confusion Matrix:\n",confusion_matrix)

num_classes = len(labels)
# Calculate TP, FP, TN, and FN for each class
for i in range(num_classes):
    TP = confusion_matrix[i, i]
    FP = tf.reduce_sum(confusion_matrix[:, i]) - TP
    FN = tf.reduce_sum(confusion_matrix[i, :]) - TP
    TN = tf.reduce_sum(confusion_matrix) - TP - FP - FN
    print(f"\nClass {i} ({labels[i]}):")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")


print("\n")
# Calculate precision, recall, and f1-score
precision = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=0)
recall = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
f1 = 2 * precision * recall / (precision + recall)

# Print precision, recall, and f1-score
for i, label in enumerate(labels):
    print(f"Label: {label}")
    print(f"Precision: {precision[i]:.3f}")
    print(f"Recall: {recall[i]:.3f}")
    print(f"F1-score: {f1[i]:.3}")

print("\n")
print("RestNet50")

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
print('Accuracy:', test_acc)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.xticks(range(len(history.history['accuracy'])))
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
print('Loss:', test_loss)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.xticks(range(len(history.history['loss'])))
plt.show()

# Get the weights and biases of the model
weights= model.get_weights()

# Print the shape of the weights and biases
print("Weights shape:", np.shape(weights))
# print("Biases shape:", np.shape(biases))

2023-04-25 07:33:01.364138: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 07:33:01.412697: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 07:33:01.413495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 07:33:02.290768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 1832 images belonging to 6 classes.
Found 1832 images belonging to 6 classes.


2023-04-25 07:33:05.680550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 69s 1s/step - loss: 2.8368 - accuracy: 0.1354
Test accuracy: 0.13537117838859558
Epoch 1/10


2023-04-25 07:34:16.448204: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


...

2023-04-25 08:02:54.493145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 66s 1s/step - loss: 0.7016 - accuracy: 0.7664
Test accuracy: 0.7663755416870117
Test loss: 0.7015868425369263




2023-04-25 08:04:00.670044: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


58/58 [==============================] - 68s 1s/step
Confusion Matrix:
 [[74 91 90 38 19 34]
 [85 78 98 48 11 27]
 [67 81 96 42  9 31]
 [62 53 65 25 10 24]
 [80 66 92 34 20 34]
 [71 56 72 19 11 19]]

Class 0 (Foi_Thong):
TP: 74, FP: 365, TN: 1121, FN: 272

Class 1 (Hang_Kra_Rog_Phu_Phan_ST1):
TP: 78, FP: 347, TN: 1138, FN: 269

Class 2 (Hang_Suea_Sakonnakhon_TT1):
TP: 96, FP: 417, TN: 1089, FN: 230

Class 3 (Kroeng_Krawia):
TP: 25, FP: 181, TN: 1412, FN: 214

Class 4 (Tanao_Si_Kan_Khaw_WA1):
TP: 20, FP: 60, TN: 1446, FN: 306

Class 5 (Tanao_Si_Kan_Sang_RD1):
TP: 19, FP: 150, TN: 1434, FN: 229


Label: Foi_Thong
Precision: 0.169
Recall: 0.214
F1-score: 0.189
Label: Hang_Kra_Rog_Phu_Phan_ST1
Precision: 0.184
Recall: 0.225
F1-score: 0.202
Label: Hang_Suea_Sakonnakhon_TT1
Precision: 0.187
Recall: 0.294
F1-score: 0.229
Label: Kroeng_Krawia
Precision: 0.121
Recall: 0.105
F1-score: 0.112
Label: Tanao_Si_Kan_Khaw_WA1
Precision: 0.250
Recall: 0.061
F1-score: 0.0985
Label: Tanao_Si_Kan_Sang_RD1


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (322,) + inhomogeneous part.

In [3]:
# Use the model to make predictions on new images
import matplotlib.pyplot as plt
import tensorflow.keras as keras

#figure 1
new_img = tf.keras.preprocessing.image.load_img('/data/test/test1.', target_size=img_size)

plt.imshow(new_img)
plt.show()

new_img = tf.keras.preprocessing.image.img_to_array(new_img)
#print(new_img)
new_img = new_img / 255.0
new_img = tf.expand_dims(new_img, 0)

prediction = model.predict(new_img)
class_index = tf.argmax(prediction, axis=1)

print('\n')
predicted_label = labels[class_index[0]]
print('Predicted label:', predicted_label)

prediction_values = {}
for i, label in enumerate(labels):
    prediction_values[label] = round(prediction[0][i], 2)

print('Prediction values:', prediction_values)


FileNotFoundError: [Errno 2] No such file or directory: '/data/test/test1.JPG'

In [1]:
print("Model Structure : \n")
model.summary()

Model Structure : 



NameError: name 'model' is not defined